# Neural Networks With TensorFlow
## Stable implementation

In [1]:
import tensorflow as tf
from tensorflow import variance_scaling_initializer # He-initializer
from jupyter_tf_graph import show_graph

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.reshape(-1, 28 * 28) / 255.0
X_test = X_test.reshape(-1, 28 * 28) / 255.0

## Training a Fully Connected Deep Neural Network
### FFNN V.01
Feed forward neural network with sigmoid activation function and Gradient Descent Optimizer

In [67]:
tf.reset_default_graph()

logs = "./tf_logs/ffnn_v01"

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.sigmoid, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden, activation=tf.nn.sigmoid, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden, activation=tf.nn.sigmoid, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden, activation=tf.nn.sigmoid, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden, activation=tf.nn.sigmoid, name="hidden5")
    outputs = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=outputs
    )
    loss = tf.reduce_mean(xentropy, name="loss")
    

alpha = 0.001
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(alpha)
    train_step = optimizer.minimize(loss)

with tf.name_scope("metrics"):
    correct = tf.nn.in_top_k(outputs, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    tb_train_writer = tf.summary.FileWriter(logs + "/train", tf.get_default_graph())
    tb_cv_writer = tf.summary.FileWriter(logs + "/cv", tf.get_default_graph())
    tb_accuracy = tf.summary.scalar("Accuracy", accuracy)
    
init = tf.global_variables_initializer()

In [68]:
%%time
epochs = 400
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        sess.run(train_step, feed_dict={X: X_train, y: y_train})
        acc, tb_acc = sess.run([accuracy, tb_accuracy], feed_dict={X: X_train, y: y_train})
        cvacc, tb_cvacc = sess.run([accuracy, tb_accuracy], feed_dict={X: X_test, y: y_test})
        tb_train_writer.add_summary(tb_acc, epoch)
        tb_cv_writer.add_summary(tb_cvacc, epoch)
        
        end = "\n" if epoch % 40 == 0 else "\r"
        print(f"@Epoch {epoch: 03}: Train Accuracy {acc:07.2%} | Test Accuracy {cvacc:07.2%}",
              end=end)

@Epoch  00: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  40: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  80: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  120: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  160: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  200: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  240: Train Accuracy 009.87% | Test Accuracy 009.80%
@Epoch  280: Train Accuracy 011.24% | Test Accuracy 011.35%
@Epoch  320: Train Accuracy 011.24% | Test Accuracy 011.35%
@Epoch  360: Train Accuracy 011.24% | Test Accuracy 011.35%
CPU times: user 24min 36s, sys: 4min 15s, total: 28min 51s%
Wall time: 6min 50s


----
### FFNN V.02
Feed forward neural network with elu activation function and Gradient Descent Optimizer

In [69]:
tf.reset_default_graph()

logs = "./tf_logs/ffnn_v02"

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden, activation=tf.nn.elu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden, activation=tf.nn.elu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden, activation=tf.nn.elu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden, activation=tf.nn.elu, name="hidden5")
    outputs = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=outputs
    )
    loss = tf.reduce_mean(xentropy, name="loss")
    

alpha = 0.001
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(alpha)
    train_step = optimizer.minimize(loss)

with tf.name_scope("metrics"):
    correct = tf.nn.in_top_k(outputs, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    tb_train_writer = tf.summary.FileWriter(logs + "/train", tf.get_default_graph())
    tb_cv_writer = tf.summary.FileWriter(logs + "/cv", tf.get_default_graph())
    tb_accuracy = tf.summary.scalar("Accuracy", accuracy)
    
init = tf.global_variables_initializer()

In [70]:
%%time
epochs = 400
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        sess.run(train_step, feed_dict={X: X_train, y: y_train})
        acc, tb_acc = sess.run([accuracy, tb_accuracy], feed_dict={X: X_train, y: y_train})
        cvacc, tb_cvacc = sess.run([accuracy, tb_accuracy], feed_dict={X: X_test, y: y_test})
        tb_train_writer.add_summary(tb_acc, epoch)
        tb_cv_writer.add_summary(tb_cvacc, epoch)
        
        end = "\n" if epoch % 40 == 0 else "\r"
        print(f"@Epoch {epoch: 03}: Train Accuracy {acc:07.2%} | Test Accuracy {cvacc:07.2%}",
              end=end)

@Epoch  00: Train Accuracy 006.13% | Test Accuracy 006.11%
@Epoch  40: Train Accuracy 009.70% | Test Accuracy 009.56%
@Epoch  80: Train Accuracy 014.78% | Test Accuracy 014.49%
@Epoch  120: Train Accuracy 021.02% | Test Accuracy 021.13%
@Epoch  160: Train Accuracy 027.68% | Test Accuracy 028.32%
@Epoch  200: Train Accuracy 033.88% | Test Accuracy 034.71%
@Epoch  240: Train Accuracy 039.29% | Test Accuracy 040.41%
@Epoch  280: Train Accuracy 043.42% | Test Accuracy 044.78%
@Epoch  320: Train Accuracy 047.05% | Test Accuracy 048.85%
@Epoch  360: Train Accuracy 050.14% | Test Accuracy 051.77%
CPU times: user 26min 10s, sys: 4min 11s, total: 30min 22s%
Wall time: 6min 56s


----
### FFNN V.03
Feed forward neural network with elu activation function and ADAM Optimizer

In [71]:
tf.reset_default_graph()

logs = "./tf_logs/ffnn_v03"

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=None, name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden, activation=tf.nn.elu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden, activation=tf.nn.elu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden, activation=tf.nn.elu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden, activation=tf.nn.elu, name="hidden5")
    outputs = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=outputs
    )
    loss = tf.reduce_mean(xentropy, name="loss")
    

alpha = 0.001
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(alpha)
    train_step = optimizer.minimize(loss)

with tf.name_scope("metrics"):
    correct = tf.nn.in_top_k(outputs, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    tb_train_writer = tf.summary.FileWriter(logs + "/train", tf.get_default_graph())
    tb_cv_writer = tf.summary.FileWriter(logs + "/cv", tf.get_default_graph())
    tb_accuracy = tf.summary.scalar("Accuracy", accuracy)
    
init = tf.global_variables_initializer()

In [72]:
%%time
epochs = 400
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        sess.run(train_step, feed_dict={X: X_train, y: y_train})
        acc, tb_acc = sess.run([accuracy, tb_accuracy],
                               feed_dict={X: X_train, y: y_train})
        cvacc, tb_cvacc = sess.run([accuracy, tb_accuracy],
                                   feed_dict={X: X_test, y: y_test})
        tb_train_writer.add_summary(tb_acc, epoch)
        tb_cv_writer.add_summary(tb_cvacc, epoch)
        
        end = "\n" if epoch % 40 == 0 else "\r"
        print(f"@Epoch {epoch: 03}: Train Accuracy {acc:07.2%} | Test Accuracy {cvacc:07.2%}",
              end=end)

@Epoch  00: Train Accuracy 025.93% | Test Accuracy 026.65%
@Epoch  40: Train Accuracy 091.63% | Test Accuracy 091.74%
@Epoch  80: Train Accuracy 094.57% | Test Accuracy 094.37%
@Epoch  120: Train Accuracy 096.32% | Test Accuracy 095.71%
@Epoch  160: Train Accuracy 097.35% | Test Accuracy 096.45%
@Epoch  200: Train Accuracy 098.10% | Test Accuracy 096.92%
@Epoch  240: Train Accuracy 098.75% | Test Accuracy 097.19%
@Epoch  280: Train Accuracy 098.96% | Test Accuracy 097.31%
@Epoch  320: Train Accuracy 099.46% | Test Accuracy 097.39%
@Epoch  360: Train Accuracy 099.69% | Test Accuracy 097.42%
CPU times: user 26min 11s, sys: 4min 17s, total: 30min 28s%
Wall time: 7min 14s


----
### FFNN V.04
Feed forward neural network with elu activation function, ADAM Optimizer and Batch Norm

In [88]:
tf.reset_default_graph()

logs = "./tf_logs/ffnn_v04/"

n_inputs = 28 * 28
n_hidden = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

eta = 0.9
with tf.name_scope("dnn"):
    z1 = tf.layers.dense(X, n_hidden, name="z1")
    z1_bn = tf.layers.batch_normalization(z1, training=training, momentum=eta)
    a1 = tf.nn.elu(z1_bn)
    
    z2 = tf.layers.dense(a1, n_hidden, name="z2")
    z2_bn = tf.layers.batch_normalization(z2, training=training, momentum=eta)
    a2 = tf.nn.elu(z2_bn)
    
    z3 = tf.layers.dense(a2, n_hidden, name="z3")
    z3_bn = tf.layers.batch_normalization(z3, training=training, momentum=eta)
    a3 = tf.nn.elu(z3_bn)
    
    z4 = tf.layers.dense(a3, n_hidden, name="z4")
    z4_bn = tf.layers.batch_normalization(z4, training=training, momentum=eta)
    a4 = tf.nn.elu(z4)
    
    z5 = tf.layers.dense(a4, n_hidden, name="z5")
    z5_bn = tf.layers.batch_normalization(z5, training=training, momentum=eta)
    a5 = tf.nn.elu(z5_bn)
    
    output = tf.layers.dense(a5, n_hidden, name="output")


with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y,
        logits=output
    )
    loss = tf.reduce_mean(xentropy, name="loss")

    
alpha = 0.001
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(alpha)
    train_step = optimizer.minimize(loss)

    
with tf.name_scope("metrics"):
    correct = tf.nn.in_top_k(output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    tb_train_writer = tf.summary.FileWriter(logs + "/train", tf.get_default_graph())
    tb_cv_writer = tf.summary.FileWriter(logs + "/cv", tf.get_default_graph())
    tb_accuracy = tf.summary.scalar("Accuracy", accuracy)

init = tf.global_variables_initializer()

In [89]:
%%time

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
epochs = 400
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        sess.run([train_step, extra_update_ops], feed_dict={training: True, X: X_train, y: y_train})
        acc, tb_acc = sess.run([accuracy, tb_accuracy],
                               feed_dict={X: X_train, y: y_train})
        cvacc, tb_cvacc = sess.run([accuracy, tb_accuracy],
                                    feed_dict={X: X_test, y: y_test})
        
        tb_train_writer.add_summary(tb_acc, epoch)
        tb_cv_writer.add_summary(tb_cvacc, epoch)
        
        end = "\n" if epoch % 40 == 0 else "\r"
        print(f"@Epoch {epoch: 03}: Train Accuracy {acc:07.2%} | Test Accuracy {cvacc:07.2%}",
              end=end)

@Epoch  00: Train Accuracy 005.91% | Test Accuracy 005.43%
@Epoch  40: Train Accuracy 091.78% | Test Accuracy 092.06%
@Epoch  80: Train Accuracy 095.16% | Test Accuracy 094.87%
@Epoch  120: Train Accuracy 096.90% | Test Accuracy 096.05%
@Epoch  160: Train Accuracy 097.95% | Test Accuracy 096.75%
@Epoch  200: Train Accuracy 098.79% | Test Accuracy 097.07%
@Epoch  240: Train Accuracy 099.32% | Test Accuracy 097.29%
@Epoch  280: Train Accuracy 099.68% | Test Accuracy 097.37%
@Epoch  320: Train Accuracy 099.87% | Test Accuracy 097.45%
@Epoch  360: Train Accuracy 099.95% | Test Accuracy 097.43%
CPU times: user 47min 35s, sys: 6min 25s, total: 54min7.43%
Wall time: 10min 52s


## Feed Forward Neural Network Version Comparison
![FFNNs Comparisson](./images/ffnn_vs.png)

# Storing and Reusing TF Models

In [84]:
from functools import partial

def a_plus_b(a, b): return a + b
a_plus_3 = partial(a_plus_b, b=3)
a_plus_3(5)

8

In [ ]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden = 100
n_output = 10

X = tf.placeholder(tf.float32, shape=(n_inputs, None), name="X")
y = tf.placeholder(tf.float32, shape=None, name="y")

training = tf.placeholder_with_default(False, shape=(), name="training")

batch_norm = partial()